<a href="https://colab.research.google.com/github/auzaluis/upsa_mod_202402/blob/main/personalidad/01_script_ETL_personalidad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Tema 01: Carga de datos**

### **Importando base de datos**

In [ ]:
# Google Auth
from google.colab import auth
auth.authenticate_user()

In [ ]:
# API Client
from google.auth import default
creds, _ = default()

In [ ]:
# gspread authorization
import gspread
gc = gspread.authorize(creds)

In [ ]:
# Accediendo al Google Sheet
url_personalidad = "https://docs.google.com/spreadsheets/d/1IQ_RxxTSmBKHTExlxboIRNlMov_F6RyqdcOPrflCv_w/edit?usp=sharing"
gsheets = gc.open_by_url(url_personalidad)
sheets = gsheets.worksheet("Respuestas de formulario 1").get_all_values()

In [ ]:
type(sheets)

In [ ]:
# Convirtiendo la lista en df
import pandas as pd
df = pd.DataFrame(sheets[1:], columns=sheets[0])

### **Inspección data frame**

In [ ]:
type(df)

In [ ]:
# Ver cantidad de filas y columnas
df.shape

In [ ]:
# Ver primeras filas
df.head()

In [ ]:
# Ver info general y nombre de columas
df.info()

In [ ]:
# Tipo de datos
df['Sexo'].dtype

In [ ]:
# Número de filas
len(df)

In [ ]:
# Número de columnas
len(df.columns)

### **Tema 02: Transformación de datos**

### **Valores perdidos**

#### **Identificación de los NAs**

In [ ]:
# Los NAs están strings vacíos
import numpy as np
df.replace("", np.nan, inplace=True)

In [ ]:
df['Escribe tu edad exacta'].isna().value_counts()

#### **Tratando de los NAs**

In [ ]:
df["Escribe tu edad exacta"].dtype

In [ ]:
df["Escribe tu edad exacta"] = pd.to_numeric(df["Escribe tu edad exacta"], errors="coerce")

In [ ]:
df["Escribe tu edad exacta"].dtype

In [ ]:
df["Escribe tu edad exacta"].mean()

In [ ]:
edad_promedio = df["Escribe tu edad exacta"].mean()

##### **Reemplazo por la media**

In [ ]:
# Creando df2
df2 = df.copy()

In [ ]:
# Reemplazo por la media
df2["edad2"] = df2["Escribe tu edad exacta"].fillna(edad_promedio)

In [ ]:
df2[["Escribe tu edad exacta", "edad2"]]

**Reubicando edad2**

In [ ]:
# Crear una lista con los nombres de las columnas
lista_columnas = list(df2.columns)

In [ ]:
# Quitando temporalmente edad2 de la lista
lista_columnas.remove("edad2")

In [ ]:
# Obteniendo el índice (posición) de la columna "after"
lista_columnas.index("Escribe tu edad exacta")

In [ ]:
# Insertando "edad2" después de "Escribe tu edad exacta"
lista_columnas.insert(lista_columnas.index("Escribe tu edad exacta") + 1, "edad2")

In [ ]:
# Reordenando y guardando en df2
df2 = df2[lista_columnas]

In [ ]:
df2.head()

**Creando relocate() en python**

In [ ]:
def relocate(df, columna, after):
  lista_columnas = list(df.columns)
  lista_columnas.remove(columna)
  lista_columnas.insert(lista_columnas.index(after) + 1, columna)
  return df[lista_columnas]

In [ ]:
relocate(
    df = df2,
    columna = "edad2",
    after = "Escribe tu edad exacta"
)

##### **Eliminar la fila completa**

In [ ]:
df2.shape

In [ ]:
df2 = df2.dropna()

In [ ]:
df2.shape

### **Estandarización de variables**

#### **Normalización**

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
# instanciando StandardScaler()
normalizador = StandardScaler()

In [ ]:
# normalizando
normalizador.fit_transform(df2[["edad2"]])

In [ ]:
# creando df3
df3 = df2.copy()

In [ ]:
df3["edadZ"] = normalizador.fit_transform(df3[["edad2"]])

In [ ]:
# reubicando edadZ despues de edad2
df3 = relocate(
    df = df3,
    columna = "edadZ",
    after = "edad2"
)

In [ ]:
df3[["edad2", "edadZ"]]

#### **Rango**

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# instanciando MinMaxScaler()
rango = MinMaxScaler()

In [ ]:
# convirtiendo a rango
df3["edadRango"] = rango.fit_transform(df3[["edad2"]])

In [ ]:
df3 = relocate(
    df = df3,
    columna = "edadRango",
    after = "edadZ"
)

In [ ]:
df3[["edad2", "edadRango"]]

### **Agrupaciones**

#### **Numéricas**

In [ ]:
cortes = [-float('inf'), 18, 21, float('inf')]
etiquetas = ["18 o menos", "19 a 21", "Más de 21"]

In [ ]:
df3['edadGR'] = pd.cut(
    x = df3['edad2'],
    bins = cortes,
    labels = etiquetas
)

In [ ]:
df3 = relocate(
    df = df3,
    columna = "edadGR",
    after = "edad2"
)

In [ ]:
# frecuencias de edadGR
df3['edadGR'].value_counts()

#### **Categóricas**

In [ ]:
# Índices
df3.info()

In [ ]:
# Valores únicos
pd.unique(df3.iloc[:,8])

In [ ]:
# Frecuencias
df3.iloc[:,8].value_counts()

In [ ]:
df3.iloc[:,8].isin(['Totalmente verdadero', 'Un poco verdadero'])

Dummitización

In [ ]:
# Función condicional normal
def top2box(x):
  if x == "Un poco verdadero" or x == "Totalmente verdadero":
    return 1
  else:
    return 0

In [ ]:
# aplicando la función
df3.iloc[:,8].apply(top2box).value_counts()

In [ ]:
# función lambda
df3.iloc[:,8].apply(lambda x: 1 if x == "Un poco verdadero" or x == "Totalmente verdadero" else 0)

In [ ]:
# frecuencias
df3.iloc[:,8].apply(lambda x: 1 if x == "Un poco verdadero" or x == "Totalmente verdadero" else 0).value_counts()

In [ ]:
df3.iloc[:,8].apply(lambda x: 1 if x in ["Un poco verdadero", "Totalmente verdadero"] else 0).value_counts()

#### **Ordinales**

In [67]:
import numpy as np

In [68]:
# niveles
pd.unique(df3.iloc[:,8])

array(['No lo sé', 'Totalmente verdadero', 'Un poco falso',
       'Un poco verdadero', 'Totalmente falso'], dtype=object)

In [69]:
# Cuando las condiciones son muchas (5 en este caso), es mejor usar select()
condiciones = [
    df3.iloc[:,8] == "Totalmente falso",
    df3.iloc[:,8] == "Un poco falso",
    df3.iloc[:,8] == "No lo sé",
    df3.iloc[:,8] == "Un poco verdadero",
    df3.iloc[:,8] == "Totalmente verdadero"
]

# Hacer coincidir el orden
opciones = [1,2,3,4,5]

In [71]:
# usando select
ordinal = np.select(
    condlist = condiciones,
    choicelist = opciones,
    default = np.nan
)

In [72]:
ordinal

array([3., 3., 5., 5., 5., 5., 5., 5., 3., 2., 5., 3., 4., 5., 4., 5., 5.,
       4., 2., 4., 2., 5., 5., 2., 5., 4., 5., 4., 5., 3., 5., 5., 5., 4.,
       5., 5., 4., 5., 5., 2., 3., 5., 5., 5., 5., 4., 5., 5., 5., 5., 5.,
       5., 5., 3., 4., 4., 5., 5., 5., 4., 5., 5., 5., 5., 5., 3., 5., 5.,
       4., 5., 4., 5., 4., 3., 4., 4., 4., 5., 4., 5., 5., 5., 4., 4., 4.,
       2., 5., 3., 5., 4., 3., 4., 3., 5., 5., 4., 5., 5., 5., 5., 5., 5.,
       5., 5., 5., 4., 5., 5., 5., 5., 5., 4., 5., 5., 5., 5., 5., 5., 5.,
       4., 5., 4., 4., 5., 4., 5., 4., 5., 5., 5., 5., 5., 5., 3., 5., 5.,
       4., 5., 4., 3., 5., 4., 4., 5., 4., 5., 5., 3., 4., 5., 4., 5., 5.,
       5., 4., 5., 4., 4., 4., 5., 5., 4., 5., 5., 4., 4., 5., 5., 3., 5.,
       5., 4., 5., 5., 5., 5., 5., 5., 5., 5., 5., 3., 4., 2., 1., 4., 5.,
       3., 3., 2., 5., 4., 4., 3., 2., 5., 3., 4., 3., 2., 5., 3., 4., 5.,
       5., 4., 3., 5., 4., 5., 2., 5., 5., 1., 4., 4., 3., 5., 3., 5., 4.,
       5., 4., 5., 5., 3.